# Python - Logging

- [Understanding pythons logging module](https://www.electricmonk.nl/log/2017/08/06/understanding-pythons-logging-module/)
- [System Development with Python - Logging and the logging module](http://uwpce-pythoncert.github.io/SystemDevelopment/logging.html)
- [Documentation - Logging configuration](https://docs.python.org/3.6/library/logging.config.html#logging-config-dictschema)
- [python logging配置時間或大小輪轉](http://www.361way.com/python-logging-timedrotating/5043.html)
- [Python 3 Logging using DictConfig](https://simpletutorials.com/c/1457/Python+3+Logging+using+DictConfig)
- [python(logging )日誌模塊學習](https://segmentfault.com/a/1190000003008066)
- [Logging Exceptions](https://stackoverflow.com/questions/5191830/how-do-i-log-a-python-error-with-debug-information)
- [Python Logging: An In-Depth Tutorial](https://www.toptal.com/python/in-depth-python-logging)
- [StackOverflow - Using Python logging in multiple modules
](https://stackoverflow.com/questions/15727420/using-python-logging-in-multiple-modules)

## Basic Concept

- **Formatter**: 設置日誌信息最後的規則、結構和內容，默認的時間格式為 `%Y-%m-%d %H:%M:%S`
- **Handler**: 將(Logger創建的)日誌記錄發送到合適的目的輸出
- **Filter**: 提供了細度設備來決定輸出哪條日誌記錄，調用 `logging.getLogger()` 時參數的格式類似於 `"A.B.C"`，採取這樣的格式其實就是為了可以配置過濾器
- **Logger**: 提供了應用程序可以直接使用的接口
- **Level**: When log level is 0, it look up the hierarchy for the first logger with a non-zero level.

| Number | Level | When it’s used |
|----:|:---|:----|
| 0 | NOTSET | Look up the hierarchy for the first logger with a non-zero level. |
| 10 | DEBUG | Detailed information, typically of interest only when diagnosing problems. |
| 20 | INFO | Confirmation that things are working as expected. |
| 30 | WARNING | An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected. |
| 40 | ERROR | Due to a more serious problem, the software has not been able to perform some function. |
| 50 | CRITICAL | A serious error, indicating that the program itself may be unable to continue running. |

![python-logging](./img/python-logging.png)

## TD;DR

- All loggers are descendants of the `root logger`.
- Don't log directly against the `root logger`. That means: no `logging.basicConfig()` or `logging.getLogger()` and no usage of module-level loggers such as `logging.warning()`, as they have unintended side-effects (no output, or double lines, etc.). (see [Appendix](#Appendix))
- The easiest way that's usually correct is to use `__name__` as the logger name: `logger = logging.getLogger(__name__)`. This uses the module hierarchy as the name, which is generally what you want. 

## Best practices

```
Project 
 ├── pkg01
 │   └── test01.py
 ├-─ pkg02
 │   └── test02.py
 └-- logging.conf
```

Best practice is, in each module, to have a logger defined near the top of the module like this:

```py
# test01.py
import logging
logger = logging.getLogger(__name__)
```

and then in other code in the module do e.g.

```py
# test01.py
logger.debug('My message with %s', 'variable data')
```

In your main program or `__init__.py` file, do:

```py
# main.py
def main():
    """your program code"""

if __name__ == '__main__':
    import logging.config
    logging.config.fileConfig('/path/to/logging.conf')
    main()
```

## Get Started

**Custom logger**

```python
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to console_handler
console_handler.setFormatter(formatter)

# add console_handler to logger
logger.addHandler(console_handler)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warn('warn message')
logger.error('error message')
logger.critical('critical message')
```

**Root logger**

> Never use root logger!!!

```python
import logging

logging.basicConfig(
    level=logging.DEBUG,  
    format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
    datefmt='%a, %d %b %Y %H:%M:%S',
    filename='/tmp/test.log',
    filemode='w'  # 默認值為 "a"，還可指定為 "w"
)

logging.debug('debug message')  
logging.info('info message')  
logging.warning('warning message')  
logging.error('error message')  
logging.critical('critical message')  
```

What does basicConfig do for you?

```python
filename = 'example.log'
filemode = 'w'
handler = logger.FileHandler(filename, mode)
format_str = '%(asctime)s %(message)s'
fmt = logger.Formatter(format_str)
handler.setFormatter(fmt)
logging.root.addHandler(h)
logging.root.setLevel(logging.EBUG)
```

## Logging hierarchy

A good convention to use when naming loggers is to use a module-level logger, in each module which uses logging, named as follows:

```python
logger = logging.getLogger(__name__)
```

Multiple calls to [`getLogger()`](https://docs.python.org/3.6/library/logging.html#logging.getLogger "logging.getLogger") with the same name will return a *reference* to the same logger object. Loggers that are further down in the hierarchical list are children of loggers higher up in the list. For example, given a logger with a name of `foo`, loggers with names of`foo.bar`, `foo.bar.baz`, and `foo.bam` are all descendants of `foo`.

**Debug the logging**

(weird logging problems such as no output, or double lines)

```python
log_to_debug = logging.getLogger("myapp.ui.edit")
while log_to_debug is not None:
    print(f'level: {log_to_debug.level} - name: {log_to_debug.name} - handlers: {log_to_debug.handlers}')
    log_to_debug = log_to_debug.parent
```

which outputs:

```
level: 0 - name: myapp.ui.edit - handlers: []
level: 0 - name: myapp.ui - handlers: []
level: 0 - name: myapp - handlers: []
level: 30 - name: root - handlers: []
```

## Dict logger settings

```python
# logging_config.py

config = {
    'version': 1,  # int representing the schema version. The only valid value at present is 1
    'formatters': {
        'default': {
            'format': '%(asctime)s - %(levelname)s - %(message)s', 
            'datefmt': '%Y-%m-%d %H:%M:%S'
        }
    },
    'handlers': {
        'console': {
            'level': 'DEBUG',  # default 'WARNING'
            'class': 'logging.StreamHandler',
            'formatter': 'default',
            'stream': 'ext://sys.stdout'
        },
        'file': {
            'level': 'DEBUG',  # default 'WARNING'
            'class': 'logging.handlers.RotatingFileHandler',
            'formatter': 'default',
            'filename': log_path,
            'maxBytes': 1024,
            'backupCount': 3
        }
    },
    'loggers': {
        'default': {
            'level': level,  # default 'WARNING'
            'handlers': ['console', 'file']
        }
    },
    'disable_existing_loggers': False
}
```

```python
import logging
import logging.config
from logging_config import config


def configure_logger(name, log_path, level):
    logging.config.dictConfig(config)
    return logging.getLogger(name)

logger = configure_logger('default', 'log13.txt', 'DEBUG')
logger.debug('debug message!')
logger.info('info message!')
logger.error('error message')
logger.critical('critical message')
logger.warning('warning message')
```

This logs to both the console and a log file. Here's the output:

```
# output
2014-11-04 23:57:44 - DEBUG - debug message!
2014-11-04 23:57:44 - INFO - info message!
2014-11-04 23:57:44 - ERROR - error message
2014-11-04 23:57:44 - CRITICAL - critical message
2014-11-04 23:57:44 - WARNING - warning message
```

## Formatters

```
%(name)s       Logger的名字
%(levelno)s    數字形式的日誌級別
%(levelname)s  文本形式的日誌級別
%(pathname)s   調用日誌輸出函數的模塊的完整路徑名，可能沒有
%(filename)s   調用日誌輸出函數的模塊的文件名
%(module)s     調用日誌輸出函數的模塊名
%(funcName)s   調用日誌輸出函數的函數名
%(lineno)d     調用日誌輸出函數的語句所在的代碼行
%(created)f    當前時間，用UNIX標準的表示時間的浮 點數表示
%(relativeCreated)d	 輸出日誌信息時的，自Logger創建以 來的毫秒數
%(asctime)s    字符串形式的當前時間。默認格式是 「2003-07-08 16:49:45,896」。逗號後面的是毫秒
%(thread)d     線程ID。可能沒有
%(threadName)s 線程名。可能沒有
%(process)d    進程ID。可能沒有
%(message)s    用戶輸出的消息
```

## Levels

```python
logger=logging.getLogger("chat.gui.statistic")
f_chat_gui = logging.Filter("chat.gui")
h_console = logging.StreamHandler()

logger.addHandler(h_console)
h_console.addFilter(f_chat_gui)
```

在Handler上添加了一個過濾器，現在我們輸出日誌信息的時候就會經過過濾器的處理。名為 `"chat.gui"` 的過濾器只讓名字帶有 `"chat.gui"` 前綴的 Logger 輸出信息。可以添加多個過濾器，只要有一個過濾器拒絕，日誌信息就不會被輸出。另外，在Logger中也可以添加過濾器。


---

## Others


### How Logging Exceptions?

`logger.exception` will output a stack trace alongside the error message.

For example:

```python
import logging
try:
    1/0
except Exception as e:
    logging.exception("message")
```

Output:

```sh
ERROR:root:message
Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
ZeroDivisionError: integer division or modulo by zero
```

## Appendix

In [1]:
import logging

# 創建一個logger  
logger = logging.getLogger()

logger1 = logging.getLogger('mylogger')
logger1.setLevel(logging.DEBUG)

logger2 = logging.getLogger('mylogger')
logger2.setLevel(logging.INFO)

logger3 = logging.getLogger('mylogger.child1')
logger3.setLevel(logging.WARNING)

logger4 = logging.getLogger('mylogger.child1.child2')
logger4.setLevel(logging.DEBUG)

logger5 = logging.getLogger('mylogger.child1.child2.child3')
logger5.setLevel(logging.DEBUG)

# 創建一個handler，用於寫入日誌文件  
fh = logging.FileHandler('/tmp/test.log')

# 再創建一個handler，用於輸出到控制台  
ch = logging.StreamHandler()

# 定義handler的輸出格式formatter  
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)

#定義一個filter
#filter = logging.Filter('mylogger.child1.child2')
#fh.addFilter(filter)  

# 給logger添加handler  
#logger.addFilter(filter)
logger.addHandler(fh)
logger.addHandler(ch)

#logger1.addFilter(filter)
logger1.addHandler(fh)
logger1.addHandler(ch)

logger2.addHandler(fh)
logger2.addHandler(ch)

#logger3.addFilter(filter)
logger3.addHandler(fh)
logger3.addHandler(ch)

#logger4.addFilter(filter)
logger4.addHandler(fh)
logger4.addHandler(ch)

logger5.addHandler(fh)
logger5.addHandler(ch)

# 記錄一條日誌  
logger.debug('logger debug message')
logger.info('logger info message')
logger.warning('logger warning message')
logger.error('logger error message')
logger.critical('logger critical message')

logger1.debug('logger1 debug message')
logger1.info('logger1 info message')
logger1.warning('logger1 warning message')
logger1.error('logger1 error message')
logger1.critical('logger1 critical message')

logger2.debug('logger2 debug message')
logger2.info('logger2 info message')
logger2.warning('logger2 warning message')
logger2.error('logger2 error message')
logger2.critical('logger2 critical message')

logger3.debug('logger3 debug message')
logger3.info('logger3 info message')
logger3.warning('logger3 warning message')
logger3.error('logger3 error message')
logger3.critical('logger3 critical message')

logger4.debug('logger4 debug message')
logger4.info('logger4 info message')
logger4.warning('logger4 warning message')
logger4.error('logger4 error message')
logger4.critical('logger4 critical message')

logger5.debug('logger5 debug message')
logger5.info('logger5 info message')
logger5.warning('logger5 warning message')
logger5.error('logger5 error message')
logger5.critical('logger5 critical message')

2018-06-13 22:01:38,332 - root - WARNING - logger warning message
2018-06-13 22:01:38,335 - root - ERROR - logger error message
2018-06-13 22:01:38,337 - root - CRITICAL - logger critical message
2018-06-13 22:01:38,339 - mylogger - INFO - logger1 info message
2018-06-13 22:01:38,339 - mylogger - INFO - logger1 info message
2018-06-13 22:01:38,344 - mylogger - WARNING - logger1 warning message
2018-06-13 22:01:38,344 - mylogger - WARNING - logger1 warning message
2018-06-13 22:01:38,348 - mylogger - ERROR - logger1 error message
2018-06-13 22:01:38,348 - mylogger - ERROR - logger1 error message
2018-06-13 22:01:38,359 - mylogger - CRITICAL - logger1 critical message
2018-06-13 22:01:38,359 - mylogger - CRITICAL - logger1 critical message
2018-06-13 22:01:38,362 - mylogger - INFO - logger2 info message
2018-06-13 22:01:38,362 - mylogger - INFO - logger2 info message
2018-06-13 22:01:38,369 - mylogger - WARNING - logger2 warning message
2018-06-13 22:01:38,369 - mylogger - WARNING - logg

### Logging flow - from python doc

![logging_flow](https://docs.python.org/3/_images/logging_flow.png)